<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [8]:
#!pip install glob
#!pip install pandas
#!pip install requests
#!pip install datetime
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=c763342e512115b1a720a395656dac6065661f9b0d1063796e373446b44a88bd
  Stored in directory: /Users/drewbiddle/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


## Imports

Import any additional libraries you may need here.


In [124]:
import glob
import pandas as pd
from datetime import datetime
import requests
import json
import os


As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [125]:
#wget command isn't found

#application/json + bank_market_cap_1
url1 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json'

#application/json + bank_market_cap_2
url2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json'

#text/csv + exchange_rates
url3 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv'

bank_market_cap_1 = requests.get(url1).json()
bank_market_cap_2 = requests.get(url2).json()
exchange_rates = requests.get(url3).text

filepath = '/Users/drewbiddle/Desktop/Python/Coursera Projects/Python Project/Data/'

# have to save files to local desktop because wget issue
with open(filepath+'bank_market_cap_1.json','w') as file:
    json.dump(bank_market_cap_1,file)
    
with open(filepath+'bank_market_cap_2.json','w') as file:
    json.dump(bank_market_cap_2,file)

with open(filepath+'exchange_rates.csv','w') as file:
    file.write(exchange_rates)


## Extract


### JSON Extract Function

This function will extract JSON files.


In [132]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [63]:
columns=['Name','Market Cap (US$ Billion)']

In [135]:
# finds a json file and calls the extract_from_json function. Then stores the data in a pandas dataframe

def extract():
    
    #create an empty dataframe
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)'])
    
    os.chdir(filepath)
    
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
        
    return extracted_data

extracted_data = extract()

extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [149]:
# Load file exchange_rates.csv as dataframe

er_df = pd.read_csv(filepath+'exchange_rates.csv',index_col = 0)
exchange_rate = er_df.loc['GBP','Rates']


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [158]:
def transform(dataframe):
    
    # load exchange rate file
    er_df = pd.read_csv(filepath+'exchange_rates.csv',index_col = 0)
    exchange_rate = er_df.loc['GBP','Rates']
    
    # change market cap column from USD to GBP
    newmcdata = dataframe['Market Cap (US$ Billion)']*exchange_rate

    # Round the Market Cap column to 3 decimal places
    newmcdata = round(newmcdata,3)

    # rename market cap to Market Cap (GBP$ Billion)
    dataframe.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'})
    
    dataframe['Market Cap (US$ Billion)'] = newmcdata
    
    return dataframe

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [146]:
def load(dataframe, targetfile):

    # use dataframe and load it to a csv
    dataframe.to_csv(targetfile, index = False)


## Logging Function


Write the logging function <code>log</code> to log your data:


In [151]:
def log(Message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + Message + '\n')
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [159]:
# Write your code here

log("ETL Job Started")

log("Set Directory Started")
filepath = '/Users/drewbiddle/Desktop/Python/Coursera Projects/Python Project/Data/'
log("Set Directory Ended")

log("Extract Data Started")
Extracted_Data = extract()
log("Extract Data Ended")

log("Transform Data Started")
Transformed_Data = transform(Extracted_Data)
log("Transform Data Ended")

log("Load Data Started")
load(Transformed_Data,'bank_market_cap_gbp.csv')
log("Load Data Ended")

log("ETL Job Ended")


### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [161]:
# Call the function here
results = extract()

# Print the rows here
results.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [162]:
# Write your code here
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [164]:
# Write your code here
log("Transform phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [168]:
# Call the function here
results = transform(extract())

# Print the first 5 rows here
results.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Log your data <code>"Transform phase Ended"</code>


In [169]:
# Write your code here
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [170]:
# Write your code here
log("Load phase Started")

Call the load function


In [173]:
# Write your code here
load(results,'testcsvfile.csv')

Log the following `"Load phase Ended"`.


In [172]:
# Write your code here
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
